In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, set_seed
import evaluate
import numpy as np
from datasets import Dataset, concatenate_datasets, Value

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/cysh/train_dataset.csv")
val_confusing_df = pd.read_csv("/content/drive/MyDrive/cysh/validation_confusing_large.csv")
test_df = pd.read_csv("/content/drive/MyDrive/cysh/test_confusing.csv")

In [ ]:
for df in [ val_confusing_df, test_df]:
    df["Sentence"] = df["Sentence"].astype(str)
    df["Sentence"] = df["Sentence"].fillna("")

train_df["text"] = train_df["text"].astype(str)
train_df["text"] = train_df["text"].fillna("")



label_map = {"negative": 0, "positive": 1}

val_confusing_df["label"] = val_confusing_df["label"].map(label_map)
test_df["label"] = test_df["label"].map(label_map)

train_dataset = Dataset.from_pandas(train_df)
val_confusing_dataset = Dataset.from_pandas(val_confusing_df)
test_dataset = Dataset.from_pandas(test_df)

train_dataset = train_dataset.cast_column("label", Value("int64"))
val_confusing_dataset = val_confusing_dataset.cast_column("label", Value("int64"))
test_dataset = test_dataset.cast_column("label", Value("int64"))

full_train_dataset = concatenate_datasets([train_dataset, val_confusing_dataset])

splits = full_train_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = splits["train"]
val_dataset = splits["test"]


model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["Sentence"], padding="max_length", truncation=True, max_length=128)



Casting the dataset:   0%|          | 0/27804 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/352 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/156 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
if "text" in train_dataset.column_names:
    train_dataset = train_dataset.remove_columns("Sentence")
    train_dataset = train_dataset.rename_column("text", "Sentence")
if "text" in val_dataset.column_names:
    val_dataset = val_dataset.remove_columns("Sentence")
    val_dataset = val_dataset.rename_column("text", "Sentence")
if "text" in test_dataset.column_names:
    test_dataset = test_dataset.rename_column("text", "Sentence")



In [ ]:
def clean_sentences(example):
    s = example["Sentence"]
    if s is None:
        s = ""
    return {"Sentence": str(s)}

train_dataset = train_dataset.map(clean_sentences)
val_dataset = val_dataset.map(clean_sentences)
test_dataset = test_dataset.map(clean_sentences)

Map:   0%|          | 0/25340 [00:00<?, ? examples/s]

Map:   0%|          | 0/2816 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

In [ ]:
train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/25340 [00:00<?, ? examples/s]

Map:   0%|          | 0/2816 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=preds, references=labels)
    f1_score = f1.compute(predictions=preds, references=labels, average="weighted")
    return {"accuracy": acc["accuracy"], "f1": f1_score["f1"]}

training_args = TrainingArguments(
    output_dir="./bert_confusing_sentiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    seed=42,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-2174991492.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
test_results = trainer.evaluate(test_dataset)

In [ ]:
test_results

{'eval_loss': 0.7339214086532593,
 'eval_model_preparation_time': 0.0187,
 'eval_accuracy': 0.5256410256410257,
 'eval_f1': 0.36220642102995043,
 'eval_runtime': 1.1821,
 'eval_samples_per_second': 131.963,
 'eval_steps_per_second': 4.23}

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,F1
1,0.296300,0.250403,0.018700,0.895241,0.895675
2,0.193100,0.303152,0.018700,0.893111,0.892601
3,0.110500,0.429841,0.018700,0.890980,0.891371


TrainOutput(global_step=4752, training_loss=0.215006491953275, metrics={'train_runtime': 1831.4818, 'train_samples_per_second': 41.507, 'train_steps_per_second': 2.595, 'total_flos': 5000425607116800.0, 'train_loss': 0.215006491953275, 'epoch': 3.0})

In [ ]:
test_results = trainer.evaluate(test_dataset)

In [ ]:
test_results

{'eval_loss': 0.8752198815345764,
 'eval_model_preparation_time': 0.0187,
 'eval_accuracy': 0.532051282051282,
 'eval_f1': 0.37638429081467056,
 'eval_runtime': 1.1238,
 'eval_samples_per_second': 138.818,
 'eval_steps_per_second': 4.449,
 'epoch': 3.0}